In [1]:
%pylab inline
import numexpr as ne
import seaborn as sns
from ipywidgets import interact
import time

Populating the interactive namespace from numpy and matplotlib


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
ne.set_num_threads(32)

8

In [4]:
import sys
sys.path.append('../../..')

In [5]:
%gui qt

In [7]:
from spiketag.mvc import Sorter

In [8]:
nCh=32
fs = 25000
# old version
#app = Sorter('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie.bin', '/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie_spktag.bin')

# new version
app = Sorter('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie.bin', spktag_filename='/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie.bin.probe')

[INFO/MainProcess] load spktag file
[INFO/MainProcess] load mua data for wave view


#############  load data  ###################
/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie.bin loaded, it contains: 
30150587 * 32 points (3859275136 bytes) 
32 channels with sampling rate of 25000.0000 
1206.023 secs (20.100 mins) of data
#############################################


In [16]:
app.run()

y_bound: (-0.11530053615570068, -0.013599109835922718)
y_bound: (0.0, 4.2857142857142856)


In [ ]:
app.save()

# Test function of locate_and_highlight

In [ ]:
app.view.wave_view.set_data(app.model.mua.data[0:200,[25,26,27]])

In [ ]:
app.model.clu[26].index

In [ ]:
app.model.spktag.t[app.model.spktag.ch==31][1:2]

In [ ]:
p = app.model.spktag.t[app.model.spktag.ch==31][-10:15990]

In [ ]:
r = np.arange(9567,9768)
np.intersect1d(r,p)

In [ ]:
%time
app.view.wave_view.palette[app.view.wave_view.clu.global2local([16]).keys()[0]]

In [ ]:
app.view.wave_view.locate_and_highlight(array([53]))

# Test performance of locate_and_highlight

In [ ]:
%load_ext line_profiler

In [ ]:
%lprun -f app.view.wave_view.locate_and_highlight app.view.wave_view.locate_and_highlight(array([53]))

In [ ]:
%lprun -f app.view.wave_view.set_range  app.view.wave_view.set_range()

In [ ]:
app.model.spktag.ch

# Test mapping = app.view.scatter_view._picker._mapping

In [ ]:
mapping.map(np.array([[-0.19727965,0.27701873,0.01775252]]))[:,:2]

In [ ]:
app.view.wave_view.waves1.nrows

In [ ]:
app.view.wave_view.waves1.index

In [ ]:
npts=3
index = np.c_[np.repeat(np.tile(np.arange(1), 3), npts),
                      np.repeat(np.arange(3), 1*npts),
                      np.tile(np.arange(npts), 3)].astype(np.float32)

In [ ]:
index

In [ ]:
wav = np.array([[0, 0, 0, 1]])

In [ ]:
mapping.map(wav)

In [ ]:
mapping = app.view.wave_view.waves1.node_transform(app.view.wave_view.grid1)

In [ ]:
mapping.map(np.array([[0,0,10,1]]))[:,:2]

In [ ]:
app.view.scatter_view.fet[:,:3]

In [ ]:
app.model.fet[26].shape

In [ ]:
app.model.mua.data[1:3,[25,26,27]]

In [ ]:
app.view.wave_view.view2.camera.transform.map(app.view.wave_view.view2.camera.transform.imap(app.view.wave_view.cross.y_axis.pos))[0]

In [ ]:
pos = app.view.wave_view.cross.y_axis.pos
print pos

In [ ]:
gl_pos = app.view.wave_view.view2.camera.transform.imap(app.view.wave_view.cross.x_axis.pos)
print gl_pos

In [ ]:
t = app.view.wave_view.cross.y_axis.glpos_to_time(0.88536586)
print t

In [ ]:
n = np.ceil(t * app.view.wave_view.cross.y_axis.fs)
print n

In [ ]:
app.view.wave_view.cross.y_axis.glpos_to_time(gl_pos)

In [ ]:
app.view.wave_view.waves1.data.shape

In [ ]:
app.view.wave_view.waves1.data[1]

In [ ]:
app.view.wave_view.waves1.ggj

In [ ]:
mapping = app.view.wave_view.waves1.node_transform(app.view.wave_view.grid1)

In [ ]:
app.view.wave_view.waves1.index[:,2].shape

In [ ]:
x_pos = -1 + 2 * app.view.wave_view.waves1.index[:,2]/ (app.view.wave_view.waves1.npts - 1)

In [ ]:
y_pos = app.view.wave_view.waves1.data

In [ ]:
a = np.array([1.0 / app.view.wave_view.waves1.ncols,1.0 / app.view.wave_view.waves1.nrows]) * 0.95

In [ ]:
b = np.array([(-1 + 2 * (app.view.wave_view.waves1.index[:,0] + 0.5) / app.view.wave_view.waves1.ncols),(-1 + 2 * (app.view.wave_view.waves1.index[:,1] * app.view.wave_view.waves1.gap+ 0.5) / app.view.wave_view.waves1.nrows)])

In [ ]:
tc.w

In [ ]:
pos = np.column_stack((x_pos,y_pos))

In [ ]:
app.view.wave_view.view2.camera.transform.map((a * pos + b.T)).shape

In [ ]:
a * pos + b.T

In [ ]:
app.view.wave_view.view2.camera.transform.map((a * pos + b.T))

In [ ]:
app.view.wave_view.all_pos

# Test wave_view alone with slider

In [ ]:
from  spiketag.view import wave_view

In [ ]:
wv = wave_view(data=app.model.mua.data)

In [ ]:
wv.grid1.add_subvisual(app.view.cluster_view)

In [ ]:
@interact(i=(1000000,2000000,15))
def render(i):
    wv._render(app.model.mua.data[i:i+15000,])

In [ ]:
wv.show()